In [10]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [24]:
batchsize=100
training_data = datasets.FashionMNIST(root="../fashion_mnist", train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root="../fashion_mnist", train=False, download=True, transform=transforms.ToTensor())
train_dataloader = DataLoader(training_data, batch_size=batchsize, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batchsize, shuffle=True)

# Define Hyperparameters

In [12]:
sequence_len = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
num_epochs = 5
learning_rate = 0.01

In [13]:
class LSTM(nn.Module):
    def __init__(self, input_len, hidden_size, num_class, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_len, hidden_size, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, num_class)

    def forward(self, X):
        hidden_states = torch.zeros(self.num_layers, X.size(0), self.hidden_size)
        cell_states = torch.zeros(self.num_layers, X.size(0), self.hidden_size)
        out, _ = self.lstm(X, (hidden_states, cell_states))
        out = self.output_layer(out[:, -1, :])
        return out

In [14]:
model = LSTM(input_size, hidden_size, num_classes, num_layers)
print(model)

LSTM(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True)
  (output_layer): Linear(in_features=128, out_features=10, bias=True)
)


In [15]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [22]:
def train(num_epochs, model, train_dataloader, loss_func, optimizer):
    total_steps = len(train_dataloader)
    for epoch in range(num_epochs):
        for batch, (images, labels) in enumerate(train_dataloader):
            images = images.reshape(-1, sequence_len, input_size)

            outputs = model(images)
            loss = loss_func(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (batch+1) % 100 == 0:
                print(f"Epoch: {epoch+1}; Batch {batch+1}; Loss: {loss.item():>4f}")

In [23]:
train(num_epochs, model, train_dataloader, loss_func, optimizer)

Epoch: 1; Batch 100; Loss: 2.299914
Epoch: 1; Batch 200; Loss: 2.299645
Epoch: 1; Batch 300; Loss: 2.298419
Epoch: 1; Batch 400; Loss: 2.294570
Epoch: 1; Batch 500; Loss: 2.296913
Epoch: 1; Batch 600; Loss: 2.292645
Epoch: 2; Batch 100; Loss: 2.291686
Epoch: 2; Batch 200; Loss: 2.292403
Epoch: 2; Batch 300; Loss: 2.283821
Epoch: 2; Batch 400; Loss: 2.283202
Epoch: 2; Batch 500; Loss: 2.275430
Epoch: 2; Batch 600; Loss: 2.273543
Epoch: 3; Batch 100; Loss: 2.272033
Epoch: 3; Batch 200; Loss: 2.265540
Epoch: 3; Batch 300; Loss: 2.251768
Epoch: 3; Batch 400; Loss: 2.257634
Epoch: 3; Batch 500; Loss: 2.247160
Epoch: 3; Batch 600; Loss: 2.233296
Epoch: 4; Batch 100; Loss: 2.214099
Epoch: 4; Batch 200; Loss: 2.199948
Epoch: 4; Batch 300; Loss: 2.180619
Epoch: 4; Batch 400; Loss: 2.149138
Epoch: 4; Batch 500; Loss: 2.124537
Epoch: 4; Batch 600; Loss: 2.064153
Epoch: 5; Batch 100; Loss: 2.000690
Epoch: 5; Batch 200; Loss: 1.888535
Epoch: 5; Batch 300; Loss: 1.909998
Epoch: 5; Batch 400; Loss: 1

In [25]:
test_images, test_labels = next(iter(test_dataloader))
test_labels

tensor([0, 8, 0, 4, 2, 5, 3, 7, 0, 8, 3, 7, 4, 3, 4, 6, 8, 1, 1, 0, 4, 3, 8, 0,
        9, 3, 0, 6, 7, 9, 8, 5, 5, 3, 9, 6, 6, 4, 9, 1, 4, 1, 5, 4, 1, 6, 7, 3,
        8, 0, 0, 4, 0, 5, 5, 7, 4, 5, 4, 6, 0, 2, 9, 3, 3, 8, 9, 6, 5, 1, 9, 3,
        2, 2, 3, 8, 7, 3, 0, 2, 8, 3, 0, 0, 5, 5, 2, 7, 5, 1, 2, 6, 1, 2, 8, 9,
        2, 5, 7, 8])

In [26]:
test_outputs = model(test_images.view(-1, 28, 28))

In [29]:
predicted = torch.max(test_outputs, 1)[1]
predicted

tensor([3, 2, 3, 2, 2, 7, 3, 7, 4, 9, 3, 7, 7, 3, 2, 3, 7, 1, 1, 3, 2, 3, 9, 3,
        9, 3, 3, 0, 7, 9, 7, 7, 9, 3, 2, 4, 4, 2, 9, 1, 4, 1, 7, 4, 1, 4, 7, 3,
        7, 3, 3, 9, 3, 7, 9, 7, 9, 7, 2, 2, 4, 2, 9, 3, 3, 9, 9, 0, 7, 1, 9, 3,
        9, 9, 1, 4, 5, 3, 4, 2, 2, 3, 3, 3, 9, 5, 2, 7, 7, 1, 3, 4, 1, 2, 2, 7,
        9, 7, 7, 3])

In [30]:
correct = [1 for i in range(100) if predicted[i] == test_labels[i]]

In [32]:
precentage_correct = sum(correct)/100
precentage_correct

0.41